In [ ]:
import tensorflow as tf
import numpy as np
import os.path as osp
import sys
sys.path.append("/home/service/")
from ThreeDLAPGAN.external.LGan.src.ae_templates import mlp_architecture_ala_iclr_18, default_train_params
from ThreeDLAPGAN.external.LGan.src.autoencoder import Configuration as Conf
from ThreeDLAPGAN.external.LGan.src.point_net_ae import PointNetAutoEncoder

from  ThreeDLAPGANexternal.LGan..src.in_out import snc_category_to_synth_id, create_dir, PointCloudDataSet, \
                                        load_all_point_clouds_under_folder
from  ThreeDLAPGANexternal.LGan..src.in_out import  PointCloudDataSet
from  ThreeDLAPGAN.src.preprocessing import  smart_upsampling,smart_downsampling,read_pickle_data,pickle_data,\
                                                Train_AE,Restore_AE,get_latent,get_encode_decode
from ThreeDLAPGAN.src.graphics import get_plot
from ThreeDLAPGAN.src.class_gan import GAN
from ThreeDLAPGAN.src.pytorch_generator_disscriminator import sample_noise,Generator,Discriminator,UnitNormClipper,\
                                        g_loss,d_loss,iterate_minibatches

import tqdm
import plotly
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
top_out_dir = '../data/'                        # Use to write Neural-Net check-points etc.
top_in_dir = '../data/shape_net_core_uniform_samples_2048/' # Top-dir of where point-clouds are stored.

experiment_name = 'single_class_ae'
n_pc_points = 2048                              # Number of points per model.
bneck_size = 128                                # Bottleneck-AE size
ae_loss = 'emd'                             # Loss to optimize: 'emd' or 'chamfer'
class_name = raw_input('Give me the class name (e.g. "car"): ').lower()


In [ ]:
syn_id = snc_category_to_synth_id()[class_name]
class_dir = osp.join(top_in_dir , syn_id)
print(class_dir)
all_pc_data = load_all_point_clouds_under_folder(class_dir, n_threads=8, file_ending='.ply', verbose=True)

In [ ]:
def get_conf(experiment_name = 'AE2048',n_pc_points = 2048,bneck_size = 128,epoch=500,held_out_step = 10,z_rotate=False ):
    top_out_dir = '../data/'                        # Use to write Neural-Net check-points etc.
    top_in_dir = '../data/shape_net_core_uniform_samples_2048/' # Top-dir of where point-clouds are stored.
    ae_loss = 'emd'                             # Loss to optimize: 'emd' or 'chamfer'
    train_dir = create_dir(osp.join(top_out_dir, experiment_name))
    train_params = default_train_params()
    encoder, decoder, enc_args, dec_args = mlp_architecture_ala_iclr_18(n_pc_points, bneck_size)
    conf = Conf(n_input = [n_pc_points, 3],
                loss = ae_loss,
                training_epochs = epoch,
                batch_size = train_params['batch_size'],
                denoising = train_params['denoising'],
                learning_rate = train_params['learning_rate'],
                train_dir = train_dir,
                loss_display_step = 10,
                saver_step = train_params['saver_step'],
                z_rotate = z_rotate,
                encoder = encoder,
                decoder = decoder,
                encoder_args = enc_args,
                decoder_args = dec_args,
                held_out_step = held_out_step
               )
    conf.experiment_name = experiment_name
#     conf.held_out_step = 5              # How often to evaluate/print out loss on held_out data (if any).
    conf.save(osp.join(train_dir, 'configuration'))
    return conf

In [ ]:
# pclouds_smart2048 = read_pickle_data(input='pclouds_smart2048.pkl')
# pclouds_smart1024 = read_pickle_data(input='pclouds_smart1024.pkl')
# pclouds_smart512 = read_pickle_data(input='pclouds_smart512.pkl')

In [ ]:
def fool_downsampling(all_pc_data,pc_count=1024):
    pclouds = np.empty([all_pc_data.point_clouds.shape[0],pc_count, all_pc_data.point_clouds.shape[2]], dtype=np.float32)
    for i,pc in enumerate(all_pc_data.point_clouds):
        mask = np.zeros(pc.shape[0])
        a = np.random.choice(pc.shape[0],size = pc_count,replace=False)
        mask[a] = 1
        pclouds[i] = pc[mask.astype(bool)]
    return pclouds


In [ ]:
pclouds_smart2048 = all_pc_data.point_clouds
pclouds_smart1024 = fool_downsampling(pc_count=1024,all_pc_data=all_pc_data)
pclouds_smart512 = fool_downsampling(pc_count=512,all_pc_data=all_pc_data)

In [ ]:
# ae512 = Train_AE(train_ae = True,data = pclouds_smart512[:-1000],conf=get_conf(experiment_name = 'AE512_car_v1_0.9_128',n_pc_points = 512,bneck_size = 128 ,epoch=1000, held_out_step=10),URL = None,held_out_data = pclouds_smart512[-1000:])

In [ ]:
# ae1024 = Train_AE(train_ae = True,data = pclouds_smart1024[:-1000],conf=get_conf(experiment_name = 'AE1024_car_v1_0.9_128',n_pc_points = 1024,bneck_size = 128 ,epoch=1000, held_out_step=10),URL = None,held_out_data = pclouds_smart1024[-1000:])

In [ ]:
# ae2048 = Train_AE(train_ae = True,data = pclouds_smart2048[:-1000],conf=get_conf(experiment_name = 'AE2048_car_v1_0.9_128',n_pc_points = 2048,bneck_size = 128 ,epoch=1000, held_out_step=10),URL = None,held_out_data = pclouds_smart2048[-1000:])

In [ ]:
ae512 = Restore_AE(name ='AE512_car_v1_0.9_128',epoch = 700,conf = get_conf(experiment_name='AE512_car_v1_0.9_128',n_pc_points=512,epoch=700))
ae1024 = Restore_AE(name ='AE1024_car_v1_0.9_128',epoch = 1000,conf = get_conf(experiment_name='AE1024_car_v1_0.9_128',n_pc_points=1024,epoch=1000))
ae2048 = Restore_AE(name ='AE2048_car_v1_0.9_128',epoch = 1000,conf = get_conf(experiment_name='AE2048_car_v1_0.9_128',n_pc_points=2048,epoch=1000))

In [ ]:
get_plot(all_pc_data.point_clouds,id_of_obj=356,mode=False)

In [ ]:
start_id = int(1.0/3.0*pclouds_smart2048.shape[0])
start_id

## GAN

In [ ]:
generator = Generator(128, out_dim = 128).cuda()
discriminator = Discriminator(in_dim = 128).cuda()

In [ ]:
# _rd = Variable(torch.FloatTensor(lt)).cuda()
# noise = Variable((torch.Tensor(sample_noise(len(lt))))).cuda()

# del generator
# del discriminator

In [ ]:
# %matplotlib inline
# import matplotlib.pyplot as plt
# plt.hist(discriminator(_rd).data.cpu().numpy().ravel(), normed=True)
# plt.hist(discriminator(generator(noise)).data.cpu().numpy().ravel(), normed=True)

In [ ]:
def get_all_lt(ae,pclouds):
    lt = get_latent(ae=ae,feed_pc=pclouds[:200])
    for i in np.arange(200,pclouds.shape[0],200):
        lt =(np.concatenate((lt,get_latent(ae=ae,feed_pc=pclouds[i:i+200])),axis=0)) 
    return lt

In [ ]:
gan_1 = GAN(generator,discriminator)
lt = get_all_lt(ae512,pclouds_smart512[:-1000])
gan_1.train(lt,num_epochs=1800,TASK = 4,k_d = 5,k_g=1)

In [ ]:
generator_2 = Generator(256, out_dim = 128).cuda()
discriminator_2 = Discriminator(in_dim = 128).cuda()

In [ ]:
gan_2 = GAN(generator_2,discriminator_2)
lt_gt = get_all_lt(ae1024,pclouds_smart1024)
lt_ups = get_all_lt(ae1024,smart_upsampling(pclouds_smart512,512,num_neighbors=7))

In [ ]:
gan_2.train(data=lt_gt[:-1000],inform=lt_ups[:-1000],num_epochs=1800,TASK=4,k_d = 5,k_g=1)

In [ ]:
generator_3 = Generator(256, out_dim = 128).cuda()
discriminator_3 = Discriminator(in_dim = 128).cuda()

In [ ]:
gan_3 = GAN(generator_3,discriminator_3)
lt_gt = get_all_lt(ae2048,pclouds_smart2048)
lt_ups = get_all_lt(ae2048,smart_upsampling(pclouds_smart1024,1024,num_neighbors=7))

In [ ]:
gan_3.train(data=lt_gt[:-1000],inform=lt_ups[:-1000],num_epochs=1800,TASK=4,k_d = 5,k_g=1)

In [ ]:
their_generator = Generator(128, out_dim = 128).cuda()
their_discriminator = Discriminator(in_dim = 128).cuda()

In [ ]:
their_gan = GAN(their_generator,their_discriminator)
lt = get_all_lt(ae2048,pclouds_smart2048[:-1000])
their_gan.train(lt,num_epochs=1800,TASK = 4)

In [ ]:
def lapgan_generate(gan_1,gan_2,gan_3,ae512,ae1024,ae2048,count = 20):
    noise = Variable(torch.Tensor(sample_noise(count)).cuda())
    data_gen_1 = gan_1.generator(noise)
    pclouds_1  = ae512.decode(data_gen_1.data.cpu().numpy())
    lt_ups = get_all_lt(ae1024,smart_upsampling(pclouds_1,512,num_neighbors=7))
    
    noise = Variable(torch.cat((torch.Tensor(sample_noise(count)),torch.Tensor(lt_ups)),1).cuda())
    data_gen_2 = gan_2.generator(noise).data.cpu().numpy()+lt_ups
    pclouds_2  = ae1024.decode(data_gen_2)
    lt_ups = get_all_lt(ae2048,smart_upsampling(pclouds_2,1024,num_neighbors=7))
    
    noise = Variable(torch.cat((torch.Tensor(sample_noise(count)),torch.Tensor(lt_ups)),1).cuda())
    data_gen_3 = gan_3.generator(noise).data.cpu().numpy() + lt_ups
    return data_gen_3

In [ ]:
num_samples =pclouds_smart2048.shape[0]

In [ ]:
lt = lapgan_generate(gan_1,gan_2,gan_3,ae512,ae1024,ae2048,count=num_samples)

In [ ]:
pclouds = ae2048.decode(lt) 

In [ ]:
get_plot(pclouds,id_of_obj=0,mode=False)

In [ ]:
import numpy as np
import os.path as osp

from ThreeDLAPGAN.external.LGan.src.evaluation_metrics import minimum_mathing_distance, \
jsd_between_point_cloud_sets, coverage

from ThreeDLAPGAN.external.LGan.src.in_out import snc_category_to_synth_id,\
                                        load_all_point_clouds_under_folder

In [ ]:
noise = Variable(torch.Tensor(sample_noise(pclouds_smart2048.shape[0])).cuda())
their_pclouds = their_gan.generator(noise).data.cpu().numpy()
their_pclouds = ae2048.decode(their_pclouds)

In [ ]:
n_ref = 1000 # size of ref_pcs.
n_sam = 1000 # size of sample_pcs.

ref_pcs = pclouds_smart2048[-1000:]
print(ref_pcs.shape)
mine_mmd = []
mine_cov = []
mine_jsd = []

for it in range(5):
    
    sample_pcs = pclouds[it*1000:it*1000+1000]
    ae_loss = 'chamfer'  # Which distance to use for the matchings.
    if ae_loss == 'emd':
        use_EMD = True
    else:
        use_EMD = False  # Will use Chamfer instead.

    batch_size = 100     # Find appropriate number that fits in GPU.
    normalize = True     # Matched distances are divided by the number of 
                         # points of thepoint-clouds.

    mmd, matched_dists = minimum_mathing_distance(sample_pcs, ref_pcs, batch_size, normalize=normalize, use_EMD=use_EMD)

    cov, matched_ids = coverage(sample_pcs, ref_pcs, batch_size, normalize=normalize, use_EMD=use_EMD)

    jsd = jsd_between_point_cloud_sets(sample_pcs, ref_pcs, resolution=28)
    print mmd, cov, jsd
    mine_mmd.append(mmd)
    mine_cov.append(cov)
    mine_jsd.append(jsd)
    
print np.mean(mine_mmd),np.mean(mine_cov),np.mean(mine_jsd)

In [ ]:
n_ref = 1000 # size of ref_pcs.
n_sam = 1000 # size of sample_pcs.

ref_pcs = pclouds_smart2048[-1000:]
print(ref_pcs.shape)
their_mmd = []
their_cov = []
their_jsd = []

for it in range(5):
    
    sample_pcs = their_pclouds[it*1000:it*1000+1000]
    ae_loss = 'chamfer'  # Which distance to use for the matchings.
    if ae_loss == 'emd':
        use_EMD = True
    else:
        use_EMD = False  # Will use Chamfer instead.

    batch_size = 100     # Find appropriate number that fits in GPU.
    normalize = True     # Matched distances are divided by the number of 
                         # points of thepoint-clouds.

    mmd, matched_dists = minimum_mathing_distance(sample_pcs, ref_pcs, batch_size, normalize=normalize, use_EMD=use_EMD)

    cov, matched_ids = coverage(sample_pcs, ref_pcs, batch_size, normalize=normalize, use_EMD=use_EMD)

    jsd = jsd_between_point_cloud_sets(sample_pcs, ref_pcs, resolution=28)
    print mmd, cov, jsd
    their_mmd.append(mmd)
    their_cov.append(cov)
    their_jsd.append(jsd)
    
print np.mean(their_mmd),np.mean(their_cov),np.mean(their_jsd)

In [ ]:
print 'Mine'
print "MMD:",np.mean(mine_mmd),'COV:',np.mean(mine_cov),'JSD:',np.mean(mine_jsd)
print 'Their'
print "MMD:",np.mean(their_mmd),"COV:",np.mean(their_cov),'JSD',np.mean(their_jsd)

In [ ]:
n_ref = pclouds_smart2048.shape[0] # size of ref_pcs.
n_sam = pclouds_smart2048.shape[0] # size of sample_pcs.

ref_pcs = pclouds_smart2048
sample_pcs = pclouds

In [ ]:
ae_loss = 'chamfer'  # Which distance to use for the matchings.

if ae_loss == 'emd':
    use_EMD = True
else:
    use_EMD = False  # Will use Chamfer instead.
    
batch_size = 100     # Find appropriate number that fits in GPU.
normalize = True     # Matched distances are divided by the number of 
                     # points of thepoint-clouds.

mmd, matched_dists = minimum_mathing_distance(sample_pcs, ref_pcs, batch_size, normalize=normalize, use_EMD=use_EMD)

cov, matched_ids = coverage(sample_pcs, ref_pcs, batch_size, normalize=normalize, use_EMD=use_EMD)

jsd = jsd_between_point_cloud_sets(sample_pcs, ref_pcs, resolution=28)

# Mine 

In [ ]:
print mmd, cov, jsd

In [ ]:
print mmd, cov, jsd

In [ ]:
print mmd, cov, jsd

In [ ]:
print mmd, cov, jsd

# Their

In [ ]:
print mmd, cov, jsd

In [ ]:
print mmd, cov, jsd

# Self

In [ ]:
noise = Variable(torch.cat((torch.Tensor(sample_noise(20)),torch.Tensor(lt_ups[:20])),1).cuda())
data_gen = gan_3.generator(noise)
pclouds = ae2048.decode(data_gen.data.cpu().numpy()) 

In [ ]:
get_plot(smart_upsampling(pclouds_smart1024,1024,num_neighbors=7) ,id_of_obj=0,mode=False)

In [ ]:
get_plot(ae2048.decode(lt_gt[:20]) ,id_of_obj=0,mode=False)

In [ ]:
get_plot(pclouds,id_of_obj=1,mode=False)

In [ ]:
get_plot(ae2048.decode(lt_ups[:20]) ,id_of_obj=0,mode=False)

In [ ]:
count = 600
noise = Variable(torch.Tensor(sample_noise(count)).cuda())
data_gen_1 = gan_1.generator(noise)
pclouds_1  = ae512.decode(data_gen_1.data.cpu().numpy())
lt_ups = get_all_lt(ae1024,smart_upsampling(pclouds_1,512,num_neighbors=7))

noise = Variable(torch.cat((torch.Tensor(sample_noise(count)),torch.Tensor(lt_ups)),1).cuda())
data_gen_2 = gan_2.generator(noise).data.cpu().numpy()+lt_ups
pclouds_2  = ae1024.decode(data_gen_2)
lt_ups = get_all_lt(ae2048,smart_upsampling(pclouds_2,1024,num_neighbors=7))

noise = Variable(torch.cat((torch.Tensor(sample_noise(count)),torch.Tensor(lt_ups)),1).cuda())
data_gen_3 = gan_3.generator(noise).data.cpu().numpy() + lt_ups
pclouds_3  = ae2048.decode(data_gen_3)

In [ ]:
# 78 820
# 7 6111
obj=1

In [ ]:
get_plot(pclouds_1,id_of_obj=obj,name='tr',mode=False)

In [ ]:
get_plot(pclouds_2,id_of_obj=obj,name='tr',mode=False)

In [ ]:
get_plot(pclouds_3,id_of_obj=obj,name='tr',mode=False)

In [ ]:
get_plot(all_pc_data.point_clouds,id_of_obj=75,mode=False,name='similar')

In [ ]:
get_plot(all_pc_data.point_clouds,id_of_obj=,mode=False)

# Get nearest neighboor

In [ ]:
from sklearn.neighbors import KernelDensity

In [ ]:
kde = [KernelDensity(0.05) for i in range(7000)]

In [ ]:
for i in range(7000):
    kde[i].fit(pclouds_smart2048[i])

In [ ]:
from tqdm import tqdm
arr = [kd.score(pclouds_3[7]) for kd in tqdm(kde)]

In [ ]:
arr = np.array(arr)

In [ ]:
np.argmax(arr)

In [ ]:
i = 0
for kd in kde:
    print(kd.score(pclouds_3[0]))

In [ ]:
kde.score(pclouds_3[0])